# Import files

In [1]:
from Utils.BinancePriceFetcher import *
from statsmodels.tsa.stattools import coint
from itertools import combinations
from tqdm import tqdm



In [2]:
# Get all available tickers
response = requests.get(f"{BASE_URL}/ticker/price")
data = response.json()
tickers = [i["symbol"] for i in data]

In [3]:
def coint_test(p1, p2):
    score, p_value, _ = coint(p1, p2)
    if p_value < 0.05:
        return True
    else:
        return False


In [23]:
def get_coint_pairs(tickers, r=2, interval = '1d', start_date="2023-01-01", end_date="2023-12-31"):
    all_pairs = list(combinations(tickers, r))
    coint_pairs = []
    for pair in tqdm(all_pairs):
        try:
            price_fetcher = BinancePriceFetcher(pair) #Initialise price fetcher
            prices = price_fetcher.get_grp_historical_ohlcv(
                interval=interval,
                start_date=start_date,
                end_date=end_date
            )

            # To prevent survivorship bias
            prices = prices.dropna()

            if coint_test(prices[pair[0]], prices[pair[1]]):
                coint_pairs += [pair]
        except KeyError:
            continue
    return coint_pairs


In [ ]:
coint_pairs = get_coint_pairs(tickers)

  0%|          | 18/4896885 [00:07<474:22:50,  2.87it/s]